# Ass 2

##### setup

In [96]:
#%%
import pandas as pd
import os
import numpy as np
import statsmodels
print(os.getcwd())
data = pd.read_csv('dest.csv', 
                header=None, 
                names=['age', 'age2', 'ed',
                       'exper', 'exper2', 'nearc2',
                       'nearc4', 'nearc4a', 'nearc4b',
                       'race', 'smsa', 'south', 'wage'])
print(d.columns.values)

C:\Users\Radim\Documents\GitHub\AdvEcox2
['age' 'age2' 'ed' 'exper' 'exper2' 'nearc2' 'nearc4' 'nearc4a' 'nearc4b'
 'race' 'smsa' 'south' 'wage']


## question 1

In [111]:
#create data
rho_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
a_list = [1, 0.6, 0.3, 0.15, 0.07, 0.04, 0.02, 0]
N = 100
k = 10
Z = np.random.normal(loc = 0, scale = 1, size =(N,k))
beta_0 = 0

for a in a_list:
    for rho in rho_list:
        pi = np.zeros(k)
        pi[0] = a
        sigma = np.array([[1, rho], [rho, 1]])
        eps_v = np.random.multivariate_normal(mean = [0,0], cov = sigma, size = (N))
        eps = eps_v[:,0]
        v = eps_v[:,1]
        
        X = Z @ pi + v
        Y = X * beta_0 + eps
        Pz = Z @ np.linalg.inv(np.transpose(Z) @ Z) @ np.transpose(Z)
        
        beta_2SLS = (np.transpose(X) @ Pz @ X) ** (-1) * (np.transpose(X) @ Pz @ Y)
        beta_2SLS_var = N * (np.transpose(X) @ Pz @ X) ** (-1)
#statsmodels.sandbox.regression.gmm.IV2SLS(endog = Y, exog, instrument=None)[source]



In [109]:
hahaha

(100, 10)

In [92]:
X.shape
print(X.shape)

(100,)


In [82]:
(np.dot(Z,pi)).shape

(100,)

In [ ]:
np.array[]